In [2]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from copy import deepcopy
from qiskit import *
from qiskit.quantum_info import state_fidelity, process_fidelity, Chi, Choi, PTM, SuperOp
from qiskit.tools.qi.qi import outer

# Tomography functions
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter

The following code displays the PTM process matrix for the 1-qubit case. Additonally, we see that 12 circuits were used which goes along with the idea that the number of measurements needed to build a process matrix goes $d^{4} - d^{2}$ where d is the number of dimensions of the Hilbert Space.

In [3]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = Choi(outer(ideal_unitary.ravel(order='F')))
chi_ideal = Chi(choi_ideal)
ptm_ideal = PTM(choi_ideal)
superop_ideal = SuperOp(choi_ideal)
# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=100)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
#print(choi_ideal)
#print(chi_ideal)
print('PTM:', ptm_ideal.data)
print('Superop:', superop_ideal.data)
qpt_tomo.set_measure_basis('Pauli')
qpt_circs

PTM: [[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]]
Superop: [[ 0.5+0.j  0.5+0.j  0.5+0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j]
 [ 0.5+0.j  0.5+0.j -0.5-0.j -0.5-0.j]
 [ 0.5+0.j -0.5+0.j -0.5-0.j  0.5+0.j]]


In [5]:
def getUnitary(circ):
    job = execute(circ, backend = BasicAer.get_backend('unitary_simulator'))
    return job.result().get_unitary(circ, decimals=3)

In [6]:
#Qiskit Uses the pauli operators / sqrt(2) as basis vectors
def pauliDict():
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circ = QuantumCircuit(q,c)

    pauli = {}
    
    I = getUnitary(circ)
    pauli['I'] = I / 2**(1/2)
    
    circX = deepcopy(circ)
    circX.x(0)
    X = getUnitary(circX)
    pauli['X'] = X / 2**(1/2)

    circY = deepcopy(circ)
    circY.y(0)
    Y = getUnitary(circY)
    pauli['Y'] = Y / 2**(1/2)

    circZ = deepcopy(circ)
    circZ.z(0)
    Z = getUnitary(circZ)
    pauli['Z'] = Z / 2**(1/2)
    
    return pauli

The following function verifies the equation for a single qubit:
$$\rho' = \Sigma_{mn} \chi_{mn} \sigma_{m}\rho\sigma_{n}$$



In [7]:
def verifyChi(circ):
    #Get the chi Matrix QISKIT produces for a given circuit
    job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
    ideal_unitary = job.result().get_unitary(circ)
    choi_ideal = Choi(outer(ideal_unitary.ravel(order='F')))
    chi = Chi(choi_ideal).data
    
    #Organizes Pauli Matrices in a list
    pauli = list(pauliDict().values())
    
    #Basis Vectors for Single Qubit 
    rho_0 = (pauli[0] + pauli[3])*(2**(1/2)/2)
    rho_1 = (pauli[0] - pauli[3])*(2**(1/2)/2)
    rho_x = (pauli[0] + pauli[1])*(2**(1/2)/2)
    rho_y = (pauli[0] + pauli[2])*(2**(1/2)/2)
    basis = [rho_0, rho_1, rho_x, rho_y]
    
    rho_0_p = 0
    rho_1_p = 0
    rho_x_p = 0
    rho_y_p = 0
    rho_p = [rho_0_p, rho_1_p, rho_x_p, rho_y_p]
    #calculate the resulting density matrices
    for a in range (4):
        for b in range(4):
            rho_0_p += chi[a][b]*pauli[a]@basis[0]@pauli[b] 
            
            rho_1_p += chi[a][b]*pauli[a]@basis[1]@pauli[b] 
            
            rho_x_p += chi[a][b]*pauli[a]@basis[2]@pauli[b] 
            
            rho_y_p += chi[a][b]*pauli[a]@basis[3]@pauli[b] 
    print(rho_0_p)
    print(rho_1_p)
    print(rho_x_p)
    print(rho_y_p)
    
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

verifyChi(circ)
            

[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.5+0.j  0. +0.5j]
 [0. -0.5j 0.5+0.j ]]
